In [1]:
# Import the python packages you will need 
import pandas 
import numpy as np
import netCDF4 as nc
import xarray as xr
import xesmf
import datetime as dt
from scipy import stats
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
# NEP Bering Grid 
bering_nep_grid_file = '/work/role.medgrp/NEP/plotting/Figure_19_20/nep_bering_grid.nc'
grd_fid = nc.Dataset(bering_nep_grid_file)

# extracting lat/lon from geolat/geolon
nep_lat = grd_fid.variables['geolat'][:]
nep_lon = grd_fid.variables['geolon'][:]

#### Interpolation for Eastern and Northern Bering Sea for Kearney spatial plotting

In [3]:
# daily NEP bering sea bottom temp
nep_fil = '/work/role.medgrp/NEP/plotting/Figure_19_20/nep_tob_bering_1993-2024_daily_ts.nc'
nep_ds = xr.open_dataset(nep_fil)
nep_ds = nep_ds.assign_coords(lon = (('yh', 'xh'), nep_lon))
nep_ds = nep_ds.assign_coords(lat = (('yh', 'xh'), nep_lat))

## daily bottom temp
#trawl_fil = './ebs_nbs_temperature_full_area.csv'
trawl_fil = '/work/role.medgrp/NEP/TOOLS/coldpool/data/ebs_nbs_temperature_full_area.csv'
df = pandas.read_csv(trawl_fil)

# drop rows that go back for forward in time further than the temporal extent of the NEP hindcast
df = df[df.year >= 1993].reset_index(drop=True)
#df = df[df.year <= 2019].reset_index(drop=True)

# only measurements with haul type 3
df = df[df.haul_type == 3].reset_index(drop=True)

# generate copy of trawl dataframe with added empty column for interpolated NEP temp
df2 = df.assign(nep_tob=np.nan*np.zeros(len(df)))

trawl_locs = xr.Dataset()
start_time = time.time()

# run interpolation step using xesmf
for row in df.itertuples():
    index=row[0]
    year = row[-1]
    
    if index%200 == 0:
        print(index,"--- %s seconds ---" % (time.time() - start_time)) # Tracking progress
    
    # get lat lon values for regridding
    trawl_locs['lon'] = xr.DataArray(data=[df['longitude'][index]], dims=('trawl_sample'))
    trawl_locs['lat'] = xr.DataArray(data=[df['latitude'][index]], dims=('trawl_sample'))
    
    # create unique regridder for trawl location
    nep_regridder = xesmf.Regridder(nep_ds, trawl_locs, 'bilinear', locstream_out=True)

    # regrid NEP and GLORYS bottom temperature value and interpolate time - Alaska Daylight Time: UTC-8
    df2['nep_tob'][index] = nep_regridder(nep_ds['tob'].interp(time=(dt.datetime.strptime(df['start_time'][index], '%Y-%m-%d %H:%M:%S')+dt.timedelta(hours=8))))
 
new_csv = 'ebs_nbs_temperature_full_area_nep.csv'
df2.to_csv(new_csv)

Could not save font_manager cache [Errno 28] No space left on device


0 --- 0.0014357566833496094 seconds ---
200 --- 578.5730018615723 seconds ---
400 --- 1114.961139202118 seconds ---


KeyboardInterrupt: 

#### Interpolation for Index Bering Sea for Rohan CPA index processing and analysis

In [ ]:
## daily bottom temp
trawl_fil = '/work/role.medgrp/NEP/TOOLS/coldpool/data/index_hauls_temperature_data.csv'
df = pandas.read_csv(trawl_fil)

# drop rows that go back for forward in time further than the temporal extent of the NEP hindcast
df = df[df.year >= 1993].reset_index(drop=True)
#df = df[df.year <= 2019].reset_index(drop=True)

# append fake 2020 time values for nep regridding
df_2020 = df[df.year==2019].reset_index(drop=True)
df_2020.year = 2020
df_2020.gear_temperature = np.nan
df_2020.surface_temperature  = np.nan
df_2020.cruise = np.nan
for row in df_2020.itertuples():
    index=row[0]
    time_str = df_2020.start_time[index].split('2019')[1] 
    df_2020.start_time[index] = ('2020' + time_str)

df = pandas.concat([df, df_2020]).reset_index(drop=True)

nep_fil = './nep_tob_bering_1993-2024_daily_ts.nc'
nep_ds = xr.open_dataset(nep_fil)
nep_ds = nep_ds.assign_coords(lon = (('yh', 'xh'), nep_lon))
nep_ds = nep_ds.assign_coords(lat = (('yh', 'xh'), nep_lat))

# generate copy of trawl dataframe with added empty column for interpolated NEP temp
df2 = df.assign(nep_tob=np.nan*np.zeros(len(df)))

trawl_locs = xr.Dataset()
start_time = time.time()

# run interpolation step using xesmf
for row in df.itertuples():
    index=row[0]
    year = row[-1]
    
    #print(dt.datetime.strptime(df['start_time'][index], '%Y-%m-%d %H:%M:%S'))
    if index%200 == 0:
        print(index,"--- %s seconds ---" % (time.time() - start_time)) # Tracking progress
    
    # get lat lon values for regridding
    trawl_locs['lon'] = xr.DataArray(data=[df['longitude'][index]], dims=('trawl_sample'))
    trawl_locs['lat'] = xr.DataArray(data=[df['latitude'][index]], dims=('trawl_sample'))
    
    # create unique regridder for trawl location
    nep_regridder = xesmf.Regridder(nep_ds, trawl_locs, 'bilinear', locstream_out=True)
    
    # regrid NEP and GLORYS bottom temperature value and interpolate time - Alaska Daylight Time: UTC-8
    df2['nep_tob'][index] = nep_regridder(nep_ds['tob'].interp(time=(dt.datetime.strptime(df['start_time'][index], '%Y-%m-%d %H:%M:%S')+dt.timedelta(hours=8))))
        
new_csv = 'index_hauls_temperature_data_nep.csv'
df2.to_csv(new_csv)